<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 9</h1>
</div>

1. Traits (§10.2)
2. Collections (§8)
3. Vectors (§8.1)
4. Tests (§11)


## 1. Traits

From [Traits: Defining Shared Behavior](https://doc.rust-lang.org/book/ch10-02-traits.html#traits-defining-shared-behavior).

* A trait defines the functionality a particular type has and can share with other types. 
* We can use traits to define shared behavior in an abstract way. 
* We can use trait bounds to specify that a generic type can be any type that has certain behavior.

> Some other programming languages call this an interface.


### Sample trait definition

Generally define method signatures as behaviors that need to implemented by
any type that implements the trait.

We can also define default implementations of default methods.

In [2]:
trait Person {
    // method header specifications -- must be implemented by any type that implements the trait
    fn get_name(&self) -> String;
    fn get_age(&self) -> u32;
    
    // default implementation of a method 
    fn description(&self) -> String {
        format!("{} ({})",self.get_name(),self.get_age())
    }
}

### Sample trait implementation 1

In [3]:
#[derive(Debug)]
struct SoccerPlayer {
    name: String,
    age: u32,
    team: String,
}

impl Person for SoccerPlayer {
    fn get_age(&self) -> u32 {
        self.age
    }
    
    // We must implement all trait items
    fn get_name(&self) -> String {
        self.name.clone()
    }
}

impl SoccerPlayer {
    fn create(name:String,age:u32,team:String) -> SoccerPlayer {
        SoccerPlayer{name,age,team}
    }
}

Since `SoccerPlayer` implements the `Person` trait, we can use the `description` method.

In [4]:
let zlatan = SoccerPlayer::create(String::from("Zlatan Ibrahimovic"), 40, String::from("AC Milan"));
println!("{}",zlatan.description());


Zlatan Ibrahimovic (40)


### Sample trait implementation 2

In [5]:
#[derive(Debug)]
struct RegularPerson {
    year_born: u32,
    first_name: String,
    middle_name: String,
    last_name: String,
}

impl Person for RegularPerson {
    fn get_age(&self) -> u32 {
        2024 - self.year_born
    }
    
    fn get_name(&self) -> String {
        if self.middle_name == "" {
            format!("{} {}",self.first_name,self.last_name)
        } else {
            format!("{} {} {}",self.first_name,self.middle_name,self.last_name)
        }
    }
}

impl RegularPerson {
    fn create(first_name:String,middle_name:String,last_name:String,year_born:u32) -> RegularPerson {
        RegularPerson{first_name,middle_name,last_name,year_born}
    }
}

In [6]:
let mlk = RegularPerson::create(
    String::from("Martin"),
    String::from("Luther"),
    String::from("King"),
    1929
);
println!("{}",mlk.description());

Martin Luther King (95)


### Using traits in functions

In [7]:
// sample function accepting object implementing trait
fn long_description(person: &impl Person) {
    println!("{}, who is {} years old", person.get_name(), person.get_age());
}

### Examples

In [8]:
long_description(&mlk);
long_description(&zlatan);

Martin Luther King, who is 95 years old
Zlatan Ibrahimovic, who is 40 years old


### Using traits in functions: long vs. short form

In [9]:
// short version
fn long_description(person: &impl Person) {
    println!("{}, who is {} old", person.get_name(), person.get_age());
}

// longer version
fn long_description_2<T: Person>(person: &T) {
    println!("{}, who is {} old", person.get_name(), person.get_age());
}


In [10]:
long_description(&zlatan);
long_description_2(&zlatan);

Zlatan Ibrahimovic, who is 40 old
Zlatan Ibrahimovic, who is 40 old


In [12]:
long_description(&mlk);
long_description_2(&mlk);


Martin Luther King, who is 95 old
Martin Luther King, who is 95 old


### So what's up with the different ways to specify traits (It's complicated!!!!)
* `&impl and &T` -> static dispatch (also relevant in the context of return values)
* `&T` restricts the type especially if you plan to pass multiple arguments of the same type (relevant to inputs)
* Read https://joshleeb.com/posts/rust-traits-and-trait-objects if you want to dig deep but without a background in programming languages and compilers this will not be possible to understand.


### Using traits in functions: multiple traits

In [13]:
use core::fmt::Debug;

fn multiple_1(person: &(impl Person + Debug)) {
    println!("{:?}",person);
    println!("Age: {}",person.get_age());
}

In [14]:
multiple_1(&zlatan);

SoccerPlayer { name: "Zlatan Ibrahimovic", age: 40, team: "AC Milan" }
Age: 40


In [15]:
multiple_1(&mlk);

RegularPerson { year_born: 1929, first_name: "Martin", middle_name: "Luther", last_name: "King" }
Age: 95


### Using traits in functions: multiple traits

In [16]:
// three options, useful for different settings

// This is good if you want to pass many parameters to the function and the parameters are of different types
fn multiple_1(person: &(impl Person + Debug)) {
    println!("{:?}",person);
    println!("Age: {}",person.get_age());
}

// This is better if you want all your parameters to be of the same type
fn multiple_2<T: Person + Debug>(person: &T) {
    println!("{:?}",person);
    println!("Age: {}",person.get_age());
}

// This is like option 2 but easier to read if your parameter combines many traits
fn multiple_3<T>(person: &T)
    where T: Person + Debug
{
    println!("{:?}",person);
    println!("Age: {}",person.get_age());
}


In [17]:
multiple_1(&mlk);
multiple_2(&mlk);
multiple_3(&mlk);

RegularPerson { year_born: 1929, first_name: "Martin", middle_name: "Luther", last_name: "King" }
Age: 95
RegularPerson { year_born: 1929, first_name: "Martin", middle_name: "Luther", last_name: "King" }
Age: 95
RegularPerson { year_born: 1929, first_name: "Martin", middle_name: "Luther", last_name: "King" }
Age: 95


### Returning types implementing a trait

In [18]:
fn get_zlatan() -> impl Person {
    SoccerPlayer::create(String::from("Zlatan Ibrahimovic"), 40, String::from("AC Milan")) 
}

In [19]:
{
    let zlatan_2 = get_zlatan();
    long_description(&zlatan_2);
};

Zlatan Ibrahimovic, who is 40 old


## 2. Collections


Rust provides a number data structures as collections.

We'll look at `Vec` today and `HashMap` later.

These are probably the most commonly used.

See [Collections](https://doc.rust-lang.org/std/collections/index.html) for the complete list.


### Why useful

* Storing multiple items, number unknown in advance

* Also the primary reason why generics exist: collections that work for different types of items

### Badly kept secret

* Most tasks: little memory management needed
* Collections will do all the work for you
* Caveat:
  - don't copy large amount of memory
  - use references

### Collection selection
* Driven by efficiency and access needs

## 3. Vectors

Expandable and shrinkable array of items:

* Python: list
* C++: vector
* Rust: vector
* Also numpy.resize() but not numpy.reshape().

**Type:** `Vec<T>` stores a collection of values of type `T`

### Creating vectors via macro `vec![...]`

In [20]:
// useful macro: vec![...]
// syntax similar to array

let small_primes = vec![2,3,5,7,11];
small_primes

[2, 3, 5, 7, 11]

In [21]:
// specifc length filled with a given value
let zeros = vec![0;10];
zeros

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [55]:
// size doesn't have to be known in advance
fn get_ones(how_many:usize) -> Vec<i32> {
    vec![1;how_many]
}

let ones = get_ones(15);
ones

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [58]:
// value and length can be specified by variables
let a = 13;
let mut b = 10;
b = 15;
let c = vec![a;b];
c

[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]

### Creating empty via the `new` constructor

In [62]:
// creating a new empty vectors
// with explicit type specification
let v1 : Vec<f64> = Vec::new();
v1

[]

In [25]:
// won't work: no type specified
let v3 = Vec::new();

Error: type annotations needed for `Vec<_>`

In [60]:
// here Rust can infer what the type is
let mut v4 = Vec::new();
let c: f32 = 13.0;
v4.push(c);  // <= add element at the end
v4

[13.0]

In [61]:
// ERROR -- infers integer then tries to push float
let mut v4 = Vec::new();
v4.push(123); // <= add element at the end
let c: f32 = 13.0;
v4.push(c);
v4

Error: mismatched types

### Basic operations

In [63]:
// adding elements at the end
let mut v2 = Vec::<bool>::new();
println!("{:?} (length={})", v2, v2.len());
v2.push(true);
v2.push(false);
println!("{:?} (length={})", v2, v2.len());

[] (length=0)
[true, false] (length=2)


In [64]:
// accessing specific element
v2[0] = v2[1];
println!("{:?}",v2);

// works because of bool values are copied
// by default

[false, false]


Try the following code which won't work as written.

In [78]:
// won't work
//#[derive(Copy, Clone)]
struct Seconds(i64);   // define a tuple struct with one element
let x = Seconds(333);
let mut v = Vec::new();
v.push(Seconds(123));
v.push(Seconds(321));
let z = v[0];

Error: cannot move out of index of `Vec<Seconds>`

There are (at least) two ways to fix the above cell.
1. Make line 8 a borrow: `let z = &v[0];` which requires another change per the error message.
2. Add `#[derive(Copy, Clone)]` to the struct definition.


In [30]:
// references do work
struct Minutes(i64);  // tuple struct with one element

{
    let mut v = Vec::new();
    v.push(Minutes(123));
    v.push(Minutes(321));
    let z1 : &Minutes = &v[1];
    let z2 : &mut Minutes = &mut v[1];
};

In [31]:
// reaching out of bounds (causes runtime panic)
let v = vec![1,2,3,4,5];
v[100]

thread '<unnamed>' panicked at src/lib.rs:230:2:
index out of bounds: the len is 5 but the index is 100
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::panicking::panic_bounds_check
   3: std::panic::catch_unwind
   4: _run_user_code_27
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


### Basic operations

Use `.get(index)` to access elements without panicking.


In [79]:
// With bound checking
// .get(index) returns Option<&T>
let mut v = vec![0,1,2];
// v[2]
v.get(2)


Some(2)

In [80]:
// try to access an element that doesn't exist
v.get(100)

None

In [36]:
// removing last element
// returns `Option<T>)
let last = v.pop();
println!("{:?}, {:?} {:?} {:?}", last, v, v.len(), v.capacity());

Some(2), [0, 1] 2 3


In [37]:
// push an element onto the end of the vector
let last = v.push(15);
println!("{:?}, {:?} {:?} {:?}", last, v, v.len(), v.capacity());

(), [0, 1, 15] 3 3


### Iterating over vector

* Iterating: `.iter`
* Mutable iterating: `.iter_mut`

In [88]:
let mut v = vec![2,1,0];
for z in v.iter() { // z in v.iter()
    println!("{}",z);
};

2


1
0


In [89]:
let mut v = vec![2,1,0];
for z in v { // implies z in v.iter()
    println!("{}",z);
};

2
1
0


In [91]:
let mut v = vec![2,1,0];
println!("{:?}",v);
for z in v.iter_mut() { // z in v.iter_mut()
    *z += 1;
}
println!("{:?}",v);

[2, 1, 0]
[3, 2, 1]


In [92]:
for z in &mut v { // implies z in v.iter_mut()
    *z += 1;
}
println!("{:?}",v);

[4, 3, 2]


In [93]:
// if you need index as well
for (i,z) in v.iter().enumerate() {
    println!("{}: {}",i,z);
};

0: 4
1: 3
2: 2


In [41]:
// Will not work since vectors have to obey memory ownership semantics
// (allocated in the heap)
let mut v1 = vec![2,1,0];
let mut v2 = v1;
println!("v1={:?}, v2={:?}", v1, v2);

Error: borrow of moved value: `v1`

In [42]:
let mut v1 = vec![2,1,0];
let mut v2 = v1.clone();  // clone instead of move
v1[0] = 12;
println!("v1={:?}, v2={:?}", v1, v2);

v1=[12, 1, 0], v2=[2, 1, 0]


Consider the following code:

In [102]:
fn testme() {
    let mut v1: Vec<u32> = vec![2,1,0];
    let mut v2: Vec<u32> = vec![2,4,6];

    // create a mutable reference to a mutable vector
    let mut v3: &mut Vec<u32> =  &mut v1;  // v3 and v1 point to the same vector

    v3[0] = 7;
    println!("v3={:?}", v3);
    println!("v1={:?}", v1);

    v3 = &mut v2;
    v3[0] = 12;
    println!("v3={:?}", v3);
    println!("v2={:?}", v2);
}
testme();

v3=[7, 1, 0]


v1=[7, 1, 0]
v3=[12, 4, 6]
v2=[12, 4, 6]


In [50]:
fn testme() {
    let mut v1: Vec<u32> = vec![2,1,0];  // v1 is a mutable vector of u32
    let mut v2: &mut Vec<u32> =  &mut v1; // v2 is a mutable reference to a mutable vector of u32
    v2[0] = 12;
    println!("v2={:?}", v2);  
    println!("v1={:?}", v1);  // immutable borrow
    // But the two below will not work.  Compiler is not smart enough to analyze what is happening inside println
    // So it appears like you are passing two references one that is immutable (v1) and one mutable (v2)
    // which is not safe.
    println!("v2={:?}", v2);
    //println!("v1={:?}, v2={:?}", v1, v2);
}
testme();

Error: cannot borrow `v1` as immutable because it is also borrowed as mutable

In [103]:
// Multidimensional vectors
let mut v1: Vec<Vec<u32>> = vec![vec![0;20];20];
println!("{:?}", v1);

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 

## 4. Tests


* Why are tests useful?
* What is typical test to functional code ratio? 

730K lines of code in Meta proxy server, roughly 1:1 ratio of tests to actual code.  [https://github.com/facebook/proxygen](https://github.com/facebook/proxygen)

Here is an example of a set of tests.

[test example](./tests/src/main.rs)

In [ ]:
fn doubleme(inp: &Vec<f64>) -> Vec<f64> {
    let mut nv = inp.clone();
    for (i, x) in inp.iter().enumerate() {
        nv[i] = *x * 2.0;
    }
    nv
}

#[test]
fn test_doubleme_positive() {
    let v = vec![1.0, 2.0, 3.0];
    let w = doubleme(&v);
    for (x, y) in v.iter().zip(w.iter()) {
        assert_eq!(*y, 2.0 * *x, "Element is not double");
    }
}
#[test]
fn test_doubleme_negative() {
    let v = vec![-1.0, -2.0, -3.0];
    let w = doubleme(&v);
    for (x, y) in v.iter().zip(w.iter()) {
        assert_eq!(*y, 2.0 * *x, "Negative element is not double");
    }
}
#[test]
fn test_doubleme_zero() {
    let v = vec![0.0];
    let w = doubleme(&v);
    for (x, y) in v.iter().zip(w.iter()) {
        assert_eq!(*y, 2.0 * *x, "Zero element is not double");
    }
}
#[test]
fn test_doubleme_empty() {
    let v: Vec<f64> = vec![];
    let w = doubleme(&v);
    assert_eq!(w.len(), 0, "Empty Vector is not empty");
}

fn testme() {
    let v: Vec<f64> = vec![2.0, 3.0, 4.0];
    let w = doubleme(&v);
    println!("V = {:?} W = {:?}", v, w);
}
testme()

**For next time, read sections ...**

1. Memory management in vectors
2. Hash maps
3. Hash maps with custom types


## In-class Poll

https://piazza.com/class/m5qyw6267j12cj/post/176
